In [28]:
from abc import ABC, abstractmethod
from calendar import c
import numpy as np

def sigmoid(x):
  return 1/(1+np.exp(-x))


class Layer(ABC):
    @abstractmethod
    def forward(self, x):
        pass

class Input(Layer):
    def forward(self, x):
        return x   

class Activation(Layer):
    def __init__(self, act):
      self.act = act

    def forward(self,x):
      return self.act(x)

class ReLu(Activation):
    def relu (_,x):
      return np.maximum(0, x)
    
    def __init__(self):
        self.act = self.relu
   
class Dense(Layer):    
    def __init__(self, w, b):
        self.w = w
        self.b = b
    
    def forward(self, x):
        return x @ self.w + self.b
    
class Model():
    def __init__(self, layers):
      self.layers = layers

    def forward_all(self, x):
        outputs = [x]
        for i, layer in enumerate(self.layers):
           outputs.append(layer.forward(outputs[i]))

        return outputs;
    


X = np.random.random(784)
# X = np.zeros(784)
W1 = np.random.random((784, 1000))
# W1 = np.zeros((784, 1000))
# np.zeros_like(W1)
B1 = np.random.random((1, 1000))
# B1 = np.zeros((1, 1000))

W2 = np.random.random((1000, 100))
# W2 = np.zeros((1000,100))
# np.zeros_like(W2)

B2 = np.random.random((1, 100))
# B2 = np.zeros((1, 100))

W3 = np.random.random((100, 10))
# W3 = np.zeros((100,10))
# np.zeros_like(W3)

# B3 = np.random.random(10)
B3 = np.zeros((1, 10))


# Crear modelo
input = Input()
relu = ReLu()
dense1 = Dense(W1, B1)
dense2 = Dense(W2, B2)
dense3 = Dense(W3, B3)

my_first_NN = Model([input, dense1, relu, dense2, relu, dense3, relu])

print(my_first_NN.forward_all(X))



[[4942563.04336772 5115329.86223934 4815455.94964348 4518038.11708695
  5215168.22182052 4810411.88999746 4698662.35230732 5689018.18715728
  4829221.82188528 4828159.63952327]]
